## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [1]:
import numpy as np
import pandas as pd

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 69

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯ'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [2]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('../kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [3]:
import nltk
from nltk import word_tokenize

import re

WORD = re.compile(r'\w+')

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

def find_max_str(companies):
    max_str = ''
    for company in companies:
        for word in WORD.findall(company):
            if(has_letters(word)):
                max_str = word if len(max_str) < len(word) else max_str 
    return max_str

def find_index_of_substring(string, companies):
    for i in range(len(companies)):
        if(companies[i].find(string) != -1):
            return i

В данной строке артефакт, подробнее в 'Testing.ipynb'

In [4]:
del train_companies[471970]

In [9]:
def extract_features(company, test=False):
    
    # Длина слова, количество гласных/согласных, положение в слове,
    # есть ли в слове что-то помимо букв и то же самое для предыдущего слова
    num_of_features = 10
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in WORD.findall(company):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    for word in WORD.findall(company):
        if(has_letters(word)):
            if(index > 0):
                features[index][5:] = features[index - 1][:5]
            features[index][0] = index/size
            features[index][1] = len(word)/MAX_WORD
            features[index][2] = count_consonants(word)/len(word)
            features[index][3] = count_vowels(word)/len(word)
            features[index][4] = int(has_not_letters(word))
            if(word[0].isupper()):
                if(word[-1].isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return features, labels
    else:
        return features

In [6]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features = np.concatenate((features, curr_features))
            labels = np.concatenate((labels, curr_labels))
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [61]:
def transform(company, labels):
    corrected = company
    index = 0
    for word in WORD.findall(company):
        if (has_letters(word)):
            print(labels[index])
            if (labels[index] == 0):
                if(len(word) < 4):
                    corrected = corrected.replace(' ' + word + ' ', ' ' + word.lower() + ' ')
                else:
                    corrected = corrected.replace(word, word.lower())
            elif(labels[index] == 1):
                if(len(word) < 4):
                    corrected = corrected.replace(' ' + word + ' ', ' ' + word[0] + word[1:].lower() + ' ')
                else:
                    corrected = corrected.replace(word, word[0] + word[1:].lower())
            index += 1
    return corrected

In [10]:
X_train, y_train = generate_dataset(train_companies[:100000])

In [11]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier()
clf.fit(X_train, y_train)

Learning rate set to 0.109939
0:	learn: 0.9850376	total: 224ms	remaining: 3m 43s
1:	learn: 0.8991012	total: 393ms	remaining: 3m 16s
2:	learn: 0.8328286	total: 595ms	remaining: 3m 17s
3:	learn: 0.7757945	total: 768ms	remaining: 3m 11s
4:	learn: 0.7311351	total: 943ms	remaining: 3m 7s
5:	learn: 0.6908427	total: 1.12s	remaining: 3m 4s
6:	learn: 0.6574303	total: 1.29s	remaining: 3m 2s
7:	learn: 0.6292450	total: 1.46s	remaining: 3m
8:	learn: 0.6052306	total: 1.61s	remaining: 2m 57s
9:	learn: 0.5833743	total: 1.77s	remaining: 2m 54s
10:	learn: 0.5649259	total: 1.92s	remaining: 2m 52s
11:	learn: 0.5466810	total: 2.07s	remaining: 2m 50s
12:	learn: 0.5289889	total: 2.23s	remaining: 2m 48s
13:	learn: 0.5152889	total: 2.37s	remaining: 2m 47s
14:	learn: 0.5011413	total: 2.52s	remaining: 2m 45s
15:	learn: 0.4898325	total: 2.67s	remaining: 2m 44s
16:	learn: 0.4789927	total: 2.83s	remaining: 2m 43s
17:	learn: 0.4689561	total: 2.97s	remaining: 2m 42s
18:	learn: 0.4607996	total: 3.12s	remaining: 2m 41s

158:	learn: 0.3015213	total: 27.5s	remaining: 2m 25s
159:	learn: 0.3013085	total: 27.6s	remaining: 2m 25s
160:	learn: 0.3011552	total: 27.8s	remaining: 2m 25s
161:	learn: 0.3009598	total: 28s	remaining: 2m 24s
162:	learn: 0.3005966	total: 28.2s	remaining: 2m 24s
163:	learn: 0.3003553	total: 28.4s	remaining: 2m 24s
164:	learn: 0.3002016	total: 28.5s	remaining: 2m 24s
165:	learn: 0.2999207	total: 28.7s	remaining: 2m 24s
166:	learn: 0.2995965	total: 28.9s	remaining: 2m 23s
167:	learn: 0.2994833	total: 29s	remaining: 2m 23s
168:	learn: 0.2991968	total: 29.2s	remaining: 2m 23s
169:	learn: 0.2989519	total: 29.3s	remaining: 2m 23s
170:	learn: 0.2987068	total: 29.5s	remaining: 2m 23s
171:	learn: 0.2985669	total: 29.7s	remaining: 2m 22s
172:	learn: 0.2984613	total: 29.9s	remaining: 2m 22s
173:	learn: 0.2983362	total: 30s	remaining: 2m 22s
174:	learn: 0.2981949	total: 30.2s	remaining: 2m 22s
175:	learn: 0.2978928	total: 30.4s	remaining: 2m 22s
176:	learn: 0.2976861	total: 30.5s	remaining: 2m 21s

316:	learn: 0.2790344	total: 53.1s	remaining: 1m 54s
317:	learn: 0.2789004	total: 53.3s	remaining: 1m 54s
318:	learn: 0.2787980	total: 53.4s	remaining: 1m 54s
319:	learn: 0.2787540	total: 53.6s	remaining: 1m 53s
320:	learn: 0.2786385	total: 53.7s	remaining: 1m 53s
321:	learn: 0.2785141	total: 53.9s	remaining: 1m 53s
322:	learn: 0.2784155	total: 54s	remaining: 1m 53s
323:	learn: 0.2783231	total: 54.2s	remaining: 1m 53s
324:	learn: 0.2782544	total: 54.4s	remaining: 1m 52s
325:	learn: 0.2781853	total: 54.5s	remaining: 1m 52s
326:	learn: 0.2781359	total: 54.6s	remaining: 1m 52s
327:	learn: 0.2780507	total: 54.8s	remaining: 1m 52s
328:	learn: 0.2779604	total: 55s	remaining: 1m 52s
329:	learn: 0.2778167	total: 55.1s	remaining: 1m 51s
330:	learn: 0.2777213	total: 55.3s	remaining: 1m 51s
331:	learn: 0.2776795	total: 55.4s	remaining: 1m 51s
332:	learn: 0.2775572	total: 55.6s	remaining: 1m 51s
333:	learn: 0.2774991	total: 55.7s	remaining: 1m 51s
334:	learn: 0.2774143	total: 55.9s	remaining: 1m 5

472:	learn: 0.2675579	total: 1m 17s	remaining: 1m 25s
473:	learn: 0.2674952	total: 1m 17s	remaining: 1m 25s
474:	learn: 0.2674356	total: 1m 17s	remaining: 1m 25s
475:	learn: 0.2673729	total: 1m 17s	remaining: 1m 25s
476:	learn: 0.2673317	total: 1m 17s	remaining: 1m 25s
477:	learn: 0.2672949	total: 1m 17s	remaining: 1m 25s
478:	learn: 0.2672544	total: 1m 18s	remaining: 1m 24s
479:	learn: 0.2672203	total: 1m 18s	remaining: 1m 24s
480:	learn: 0.2671537	total: 1m 18s	remaining: 1m 24s
481:	learn: 0.2670903	total: 1m 18s	remaining: 1m 24s
482:	learn: 0.2670680	total: 1m 18s	remaining: 1m 24s
483:	learn: 0.2670076	total: 1m 18s	remaining: 1m 24s
484:	learn: 0.2669701	total: 1m 18s	remaining: 1m 23s
485:	learn: 0.2669521	total: 1m 19s	remaining: 1m 23s
486:	learn: 0.2669133	total: 1m 19s	remaining: 1m 23s
487:	learn: 0.2668060	total: 1m 19s	remaining: 1m 23s
488:	learn: 0.2667417	total: 1m 19s	remaining: 1m 23s
489:	learn: 0.2666936	total: 1m 19s	remaining: 1m 23s
490:	learn: 0.2666367	total:

625:	learn: 0.2600000	total: 1m 42s	remaining: 1m 1s
626:	learn: 0.2599349	total: 1m 42s	remaining: 1m
627:	learn: 0.2598783	total: 1m 42s	remaining: 1m
628:	learn: 0.2598186	total: 1m 42s	remaining: 1m
629:	learn: 0.2597999	total: 1m 43s	remaining: 1m
630:	learn: 0.2597854	total: 1m 43s	remaining: 1m
631:	learn: 0.2596977	total: 1m 43s	remaining: 1m
632:	learn: 0.2596711	total: 1m 43s	remaining: 1m
633:	learn: 0.2596322	total: 1m 43s	remaining: 59.9s
634:	learn: 0.2595470	total: 1m 43s	remaining: 59.7s
635:	learn: 0.2594893	total: 1m 44s	remaining: 59.6s
636:	learn: 0.2594675	total: 1m 44s	remaining: 59.4s
637:	learn: 0.2594516	total: 1m 44s	remaining: 59.2s
638:	learn: 0.2593811	total: 1m 44s	remaining: 59s
639:	learn: 0.2593368	total: 1m 44s	remaining: 58.9s
640:	learn: 0.2593065	total: 1m 44s	remaining: 58.7s
641:	learn: 0.2592599	total: 1m 44s	remaining: 58.5s
642:	learn: 0.2591911	total: 1m 45s	remaining: 58.4s
643:	learn: 0.2591317	total: 1m 45s	remaining: 58.2s
644:	learn: 0.25

782:	learn: 0.2539415	total: 2m 7s	remaining: 35.5s
783:	learn: 0.2539083	total: 2m 8s	remaining: 35.3s
784:	learn: 0.2538810	total: 2m 8s	remaining: 35.1s
785:	learn: 0.2538569	total: 2m 8s	remaining: 35s
786:	learn: 0.2538057	total: 2m 8s	remaining: 34.8s
787:	learn: 0.2537792	total: 2m 8s	remaining: 34.7s
788:	learn: 0.2537451	total: 2m 8s	remaining: 34.5s
789:	learn: 0.2537225	total: 2m 9s	remaining: 34.3s
790:	learn: 0.2536987	total: 2m 9s	remaining: 34.2s
791:	learn: 0.2536393	total: 2m 9s	remaining: 34s
792:	learn: 0.2535794	total: 2m 9s	remaining: 33.9s
793:	learn: 0.2535523	total: 2m 9s	remaining: 33.7s
794:	learn: 0.2535199	total: 2m 10s	remaining: 33.5s
795:	learn: 0.2535023	total: 2m 10s	remaining: 33.4s
796:	learn: 0.2534706	total: 2m 10s	remaining: 33.2s
797:	learn: 0.2534295	total: 2m 10s	remaining: 33s
798:	learn: 0.2533956	total: 2m 10s	remaining: 32.9s
799:	learn: 0.2533885	total: 2m 10s	remaining: 32.7s
800:	learn: 0.2533735	total: 2m 10s	remaining: 32.5s
801:	learn:

938:	learn: 0.2493797	total: 2m 33s	remaining: 9.94s
939:	learn: 0.2493579	total: 2m 33s	remaining: 9.78s
940:	learn: 0.2493352	total: 2m 33s	remaining: 9.62s
941:	learn: 0.2493237	total: 2m 33s	remaining: 9.45s
942:	learn: 0.2492846	total: 2m 33s	remaining: 9.29s
943:	learn: 0.2492631	total: 2m 33s	remaining: 9.12s
944:	learn: 0.2492456	total: 2m 33s	remaining: 8.96s
945:	learn: 0.2492361	total: 2m 34s	remaining: 8.79s
946:	learn: 0.2492084	total: 2m 34s	remaining: 8.63s
947:	learn: 0.2492028	total: 2m 34s	remaining: 8.47s
948:	learn: 0.2491674	total: 2m 34s	remaining: 8.3s
949:	learn: 0.2491453	total: 2m 34s	remaining: 8.14s
950:	learn: 0.2490997	total: 2m 34s	remaining: 7.98s
951:	learn: 0.2490683	total: 2m 34s	remaining: 7.81s
952:	learn: 0.2490420	total: 2m 35s	remaining: 7.65s
953:	learn: 0.2490324	total: 2m 35s	remaining: 7.49s
954:	learn: 0.2489825	total: 2m 35s	remaining: 7.32s
955:	learn: 0.2489719	total: 2m 35s	remaining: 7.16s
956:	learn: 0.2489424	total: 2m 35s	remaining: 

In [80]:
company = 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АСАВТАМАК" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН'
extract_features(company, test=True)

array([[0.        , 0.1884058 , 0.46153846, 0.46153846, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.07142857, 0.27536232, 0.47368421, 0.47368421, 0.        ,
        0.        , 0.1884058 , 0.46153846, 0.46153846, 0.        ],
       [0.14285714, 0.14492754, 0.5       , 0.5       , 0.        ,
        0.07142857, 0.27536232, 0.47368421, 0.47368421, 0.        ],
       [0.21428571, 0.13043478, 0.44444444, 0.44444444, 0.        ,
        0.14285714, 0.14492754, 0.5       , 0.5       , 0.        ],
       [0.28571429, 0.27536232, 0.47368421, 0.47368421, 0.        ,
        0.21428571, 0.13043478, 0.44444444, 0.44444444, 0.        ],
       [0.35714286, 0.07246377, 0.6       , 0.4       , 0.        ,
        0.28571429, 0.27536232, 0.47368421, 0.47368421, 0.        ],
       [0.42857143, 0.01449275, 1.        , 0.        , 0.        ,
        0.35714286, 0.07246377, 0.6       , 0.4       , 0.        ],
       [0.5       , 0.13043478, 0.5555555

In [81]:
transform(company, clf.predict(extract_features(company, test=True)))

[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]


'Муниципальное общеобразовательное учреждение "Начальная общеобразовательная школа Д.Асавтамак" муниципального района Бураевский район Республики Башкортостан'

In [79]:
test_companies[1000:1200]

['КРЕСТЬЯНСКОЕ (ФЕРМЕРСКОЕ) ХОЗЯЙСТВО "ЮПИТЕР" БУРАЕВСКОГО РАЙОНА РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МКУ ОТДЕЛ ОБРАЗОВАНИЯ АМР БР РБ',
 'МОУ НОШ Д.НАРЫШЕВО',
 'МНОШ ДЕРЕВНИ ЯКУНИНО',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АИТОВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МУНИЦИПАЛЬНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ДЕТСКИЙ САД Д.КУЗБАЕВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МОБУ СОШ ИМ.М.Г.ХАЙРУЛЛИНОЙ Д.КАШКАЛЕВО',
 'МОБУ ООШ Д.МАМАДЫ',
 'МООШ Д.КАРГАЛЫ',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АСАВТАМАК" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МОБУ НОШ №3 С.БУРАЕВО',
 'МОБУ СОШ Д. БОЛЬШЕШУКШАНОВО',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ОСНОВНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д. БОЛЬШЕШУКШАНОВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВА

In [29]:
result = open('result.txt', 'w')
for company in test_companies:
    result.write(transform(company, clf.predict(extract_features(company, test=True))) + '\n')